## Project - Seattle Airbnb Dataset  

BY Fabi Daniel

In [2]:
# import all necessary modules
import numpy as np
print('numpy: {}'.format(np.__version__))
import pandas as pd
print('pandas: {}'.format(pd.__version__))
from matplotlib import pyplot as plt
# print('matplotlib: {}'.format(matplotlib.__version__))
import seaborn as sns
print('seaborn: {}'.format(sns.__version__))
%matplotlib inline

numpy: 1.23.1
pandas: 1.4.3
seaborn: 0.11.2


In [3]:
#load dataset

calendar_df = pd.read_csv('/Users/cwrealestate/Downloads/archive 2/calendar.csv')
listings_df = pd.read_csv('/Users/cwrealestate/Downloads/archive 2/listings.csv')
reviews_df = pd.read_csv('/Users/cwrealestate/Downloads/archive 2/reviews.csv')

In [ ]:
# Get overview of Calendar
calendar_df.head()

In [ ]:
# Get info 
calendar_df.info()

* There are Four Columns In calendar_df, 1393570 observations

In [ ]:
listings_df.head()

In [ ]:
listings_df.info()

* There are 91 columns in listings_df, 3818 observations

In [ ]:
reviews_df.head()

In [ ]:
reviews_df.info()

* There are 6 columns in reviews_df, 84849 observations

### Data Assessment


In [ ]:
listings_df.sample(10)

##### Overview of listings_df

It has 91 columns

In [ ]:
# Get an Overview of columns that are float
listings_df.loc[:,(listings_df.dtypes=='float64').values]

In [ ]:
# Get an Overview of columns that are Object
listings_df.loc[:,(listings_df.dtypes=='object').values]

In [ ]:
listings_df.loc[:,(listings_df.dtypes=='int').values]

In [ ]:
listings_df.loc[:,(listings_df.dtypes=='datetime').values]

There are no datetime columns in the listings_df. Clever to have separated the datetime. 

In [ ]:
listings_df.columns

Due to the Number of columns, working with this table might be cumbersome. I will normalize the listings_df table into smaller tables with id being the key to the relationship

##### Overview of calendar_df

It has four columns

In [ ]:
calendar_df.loc[:,(calendar_df.dtypes=='datetime').values]

In [ ]:
print(calendar_df.columns)
calendar_df.info()

Column 'date' is not in right datatype

In [ ]:
calendar_df.date.dtypes

##### Overview of review_df

It has six columns

In [ ]:
reviews_df.info()

Column 'date' is not in right datatype

#### Data Cleaning and Normalization

Convert date to appropriate datatype

In [ ]:
# Convert calendar_df Date
calendar_df['date'] = pd.to_datetime(calendar_df.date, yearfirst=True)
calendar_df['date'].dtypes

In [ ]:
reviews_df['date'] = pd.to_datetime(reviews_df.date, yearfirst=True)
reviews_df['date'].dtypes

##### Check for duplicates

In [8]:
print(reviews_df.duplicated().sum())
print(calendar_df.duplicated().sum())
print(listings_df.duplicated().sum())

0
0
0


##### Data Normalization of Listing DataFrame

In [ ]:
# Normalize for listings_location for locational details

listings_location = listings_df[['id',
'name', 
'description', 
'neighbourhood', 
'city', 
'state', 
'is_location_exact',
 'zipcode',
 'market',
 'smart_location',
 'country_code',
 'country',
 'latitude',
 'longitude']]

listings_location.shape

listings_location has 9 columns 

In [ ]:
# Normalize for listings_price for pricing details
listings_price = listings_df[['id', 
'price',
'weekly_price',
'monthly_price',
'security_deposit',
'cleaning_fee']]

listings_price.shape

In [ ]:
listings_df[['guests_included','extra_people']]


In [ ]:
listings_df[['minimum_nights',                    
'maximum_nights',                       
'has_availability',                
'availability_30',                
'availability_60',                    
'availability_90',   
'availability_365']]

listings_price has 8 columns

In [ ]:
# Normalize for listings_bedtails for bedding details

listings_bedtails = listings_df[['id', 
'property_type',
'room_type',
'accommodates',
'bathrooms',
'bedrooms',
'beds',
'bed_type',
'amenities',
'square_feet',
'amenities',
'square_feet']]

listings_bedtails

listings_bedtails has 8 columns 

In [ ]:
listings_hosts = listings_df[['id', 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified']]

listings_hosts.shape

listings_hosts has 19 columns

In [ ]:
listings_review = listings_df[['id', 'number_of_reviews',
 'first_review',
 'last_review',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value']]


# listings_review
listings_review.shape

In [ ]:
listings_review.sample(10)

listings_review has 11 columns

In [ ]:
11+19+8+8+16+23

In [ ]:
type(list(listings_bedtails.columns))

In [ ]:

# for x in list2:
#   list1.append(x)



# for x in list2:
#   list1.append(x)

init_listings_col = list(listings_bedtails.columns).extend(listings_hosts.columns.values)
# type(init_listings_col)


# print(init_listings_col)
# for i in :
#     for i in listings_df.columns.to_list:
#     if 

In [ ]:
listings_others=listings_df[['guests_included',
'extra_people',
'minimum_nights',
'maximum_nights',
'calendar_updated',
'has_availability',
'availability_30',
'availability_60',
'availability_90',
'availability_365',
'calendar_last_scraped','requires_license',
'license',
'jurisdiction_names',
'instant_bookable',
'cancellation_policy',
'require_guest_profile_picture',
'require_guest_phone_verification',
'calculated_host_listings_count',
'reviews_per_month']]

listings_others.shape

### Exploratory Data Analysis

First, I will calculate the occupancy rate of each listings. Occupancy Rate is a very important metric for short stays as it shows how well your apartment is performing, a High Occupancy Rate is considered good.
* To get the Occupancy Rate for each listing, I divide the Number of days the listing was occupied by the total number of days in question
* It is worthy to note that it is possible that listings might not have the same start date. To account for this, I will get the date difference for each listing; the difference between date of first instance and date of last instance

In [ ]:
# add aggregation by first and last 
df_agg= calendar_df.groupby(['listing_id']).agg({'date':['first','last']})

# Flatten Multiindex
df_agg.columns = df_agg.columns.map('_'.join)


# Convert to Date time
df_agg[['date_first','date_last']] = df_agg[['date_first','date_last']].apply(pd.to_datetime) #if conversion required

# Find Date difference
df_agg['date_diff'] = (df_agg['date_last'] - df_agg['date_first'])
df_agg.sample(10)

In [ ]:
df_agg['date_last'].value_counts()

* All the listings have the same date difference. Now, I can proceed finally to calculating the occupancy rate using 365 days as the denominator

In [ ]:
calendar_df

In [ ]:
# Calculate the Occupancy Count i.e. (where 'available' is 't')
occupancy = calendar_df.groupby('listing_id')['available'].apply(lambda x: (x=='t').sum()).reset_index(name='count')

# Create 'occupancy_rate' feature for each Listing.
occupancy['rate'] = (occupancy['count']*100)/365
occupancy.sample(20)

In [ ]:
# Get Distribution of occupancy rate
# plt.figure(figsize=(12,7))
plt.rcParams['figure.figsize']=[12,7]
occupancy['rate'].plot(kind='hist', bins=20)
plt.title('Distribution of Occupancy Rate')
plt.show()

Apparently, there are listings with a perfect record i.e. 100% occupancy rate and this group represents the majority 

I will get a table for all listings with a perfect record.

In [ ]:
# Get a table of all listings with a perfect record

perfect_rec = occupancy[occupancy['rate'] == 100]
perfect_rec.info()

There are 678 listings with a perfect record 

I feel pressured to take a deeper dive to understand the characteristics of this particular group, but before then, I will peruse through the normalized llisting DataFrames

In [ ]:
# Check bed details 

listings_bedtails.sample(10)

In [ ]:

# Plot for Property Type
# Choose color for plot
def count_univ(data, y, color= sns.color_palette()[0]):
    order = data[y].value_counts().index

    sns.countplot(data=data, y=y, color=color, order=order)
    plt.title(f'Frequency of {y}')
    plt.ylabel(f'{y}', fontsize=12)
    plt.xlabel('Count', fontsize=12)
    plt.show()
count_univ(data=listings_bedtails,y='property_type')

In general, There are more houses than any other property type for short stay accommodation in Seattle. Following closely are Apartments.

The Margin for error is not known as it's common for owners when uploading to use a generic term 'House' for the property description



In [ ]:
# Plot for Room Type
count_univ(data=listings_bedtails,y='room_type')

As expected there are more listings let out in their entirety that there are shared or private rooms

Next is to check for bedrooms

In [ ]:
# Plot for Room Type
count_univ(data=listings_bedtails,y='bedrooms')

First thing to note from the visualization is the 0 bedrooms. This could be studio apartment that have both the bedroom and the living area as one single area.

Also, 1-Bedrooms are the most frequent bedroom type, representing majority of the dataset 

Now, merging the listing_bedtails dataframe with my perfect_rec dataframe, I can get the bed details for the perfect record listings and see how they are represented in my perfect_rec dataframe

In [ ]:
# Merge Bed Details with Perfect record
temp_listings = perfect_rec.merge(listings_bedtails, how='left',left_on='listing_id',right_on='id',suffixes=('', '_drop'))

#Drop the duplicate columns
temp_listings.drop([col for col in temp_listings.columns if 'drop' in col], axis=1, inplace=True)




Plot for bedrooms for listings with Perfect Records

In [ ]:
# base_color = sns.color_palette()[0]

# temp_listings.bedrooms.value_counts(normalize=True).plot(kind='bar', color=base_color)
# # plt.xticks(rotation=45)
# temp_listings

# Plot for bedrooms for listings with Perfect Records
count_univ(data=temp_listings,y='bedrooms')

Apart from the fact that there are no 6 or 7 Bedrooms with a Perfect Record. This chart is similar to the one for all listings

In [ ]:
# Plot for Room Type for listings with Perfect Records
count_univ(data=temp_listings,y='room_type')

This is interesting, compared to the chart for all listings, private rooms have a very good representation when it comes to having Perfect Records. 

I would love to see the ratio of Private Rooms with Perfect Records to All Private Rooms

In [ ]:
# Plot for Property Type for listings with Perfect Records
count_univ(data=temp_listings,y='property_type')

The gap between the two leading property type has increased.

However, there are still more **Houses** with Perfect Records than any other property type for short stay accommodation in Seattle. Following closely are Apartments.

The Margin for error is not known as it's common for owners when uploading to use a generic term 'House' for the property description



In [ ]:
calendar_df

In [ ]:
# df_agg['diff'].value_counts()

In [ ]:
# temp_listings = listings_max.merge(listings_bedtails, how='left',left_on='id',right_on='id',suffixes=('', '_drop'))

# #Drop the duplicate columns
# temp_listings.drop([col for col in temp_listings.columns if 'drop' in col], axis=1, inplace=True)

# base_color = sns.color_palette()[0]

# temp_listings.beds.value_counts(normalize=True).plot(kind='bar', color=base_color)
# # plt.xticks(rotation=45)
# # temp_listings


In [ ]:
reviews_df.sample(10)

In [ ]:
reviews_count = reviews_df.groupby('listing_id')['listing_id'].count().sort_values(ascending=False).reset_index(name='review_count')

In [ ]:
reviews_count

In [ ]:
# occupancy_review = occupancy.merge(reviews_count, how='left',left_on='listing_id',right_on='listing_id',suffixes=('', '_drop'))


# x= occupancy_review['review_count']
# y = occupancy_review['rate']

# bins = np.arange(x.min(),x.max(),0.5)
# sns.regplot(data=occupancy_review, x=x, y=y,scatter_kws={'alpha': 1/20}, bins=bins)